In [13]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 25.7 MB/s eta 0:00:00


In [ ]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_spli

In [3]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
X.shape,y.shape

((768, 8), (768,))

In [7]:
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(Xs,y, test_size=0.2, random_state=42)

In [9]:
LAYERS = [
    tf.keras.layers.Dense(32, activation='relu', input_dim = 8),
    tf.keras.layers.Dense(1, activation='sigmoid')
]

model = tf.keras.models.Sequential(LAYERS)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
LOSS_FUNCTION = 'binary_crossentropy'
OPTIMIZER = 'Adam'
METRICS = ['accuracy']

model.compile(loss= LOSS_FUNCTION,
              optimizer= OPTIMIZER,
              metrics= METRICS)

In [12]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 1s 14ms/step - loss: 0.6746 - accuracy: 0.5863 - val_loss: 0.6472 - val_accuracy: 0.6234
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6261 - accuracy: 0.6775 - val_loss: 0.6098 - val_accuracy: 0.7078
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5893 - accuracy: 0.7378 - val_loss: 0.5812 - val_accuracy: 0.7468
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5598 - accuracy: 0.7638 - val_loss: 0.5623 - val_accuracy: 0.7597
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5381 - accuracy: 0.7687 - val_loss: 0.5469 - val_accuracy: 0.7727
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5220 - accuracy: 0.7687 - val_loss: 0.5335 - val_accuracy: 0.7662
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5087 - accuracy: 0.7687 - val_loss: 0.5259 - val_accuracy: 0.7662
Epoch 8/10
20/20 [=

In [15]:
def build_model(hp):

  LAYERS = [
    tf.keras.layers.Dense(32, activation='relu', input_dim = 8),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ]

  model = tf.keras.models.Sequential(LAYERS)


  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  METRICS = ['accuracy']

  model.compile(loss= LOSS_FUNCTION,
                optimizer= OPTIMIZER,
                metrics= METRICS)

  return model


In [16]:
#randomSearch
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

In [17]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 16s


In [18]:
#Best optimizer
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X_train, y_train, epochs=10, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/10
20/20 [==============================] - 1s 16ms/step - loss: 0.5195 - accuracy: 0.7459 - val_loss: 0.5244 - val_accuracy: 0.7727
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5061 - accuracy: 0.7573 - val_loss: 0.5177 - val_accuracy: 0.7727
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4954 - accuracy: 0.7671 - val_loss: 0.5115 - val_accuracy: 0.7662
Epoch 10/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4870 - accuracy: 0.7720 - val_loss: 0.5078 - val_accuracy: 0.7727


In [22]:
def build_model(hp):

  units = hp.Int('units', min_value = 8, max_value = 128, step=8)

  LAYERS = [
    tf.keras.layers.Dense(units= units, activation='relu', input_dim = 8),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ]

  model2 = tf.keras.models.Sequential(LAYERS)


  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = 'rmsprop'
  METRICS = ['accuracy']

  model2.compile(loss= LOSS_FUNCTION,
                optimizer= OPTIMIZER,
                metrics= METRICS)

  return model2

In [23]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir')

In [24]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 12s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [26]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(88, activation = 'relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(88, activation = 'relu'))

  model.add(Dense(1, activation = 'sigmoid'))

  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = 'rmsprop'
  METRICS = ['accuracy']

  model.compile(loss= LOSS_FUNCTION,
                optimizer= OPTIMIZER,
                metrics= METRICS)

  return model

In [27]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir1')

In [28]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 18s


In [29]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [30]:
model3 = tuner.get_best_models(num_models=1)[0]

In [31]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 88)                792       
                                                                 
 dense_1 (Dense)             (None, 88)                7832      
                                                                 
 dense_2 (Dense)             (None, 88)                7832      
                                                                 
 dense_3 (Dense)             (None, 88)                7832      
                                                                 
 dense_4 (Dense)             (None, 1)                 89        
                                                                 
Total params: 24377 (95.22 KB)
Trainable params: 24377 (95.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


HYPERPARAMETER TUNNING BUILD FUNCTION

In [32]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value= 1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh']),
                      input_dim=8))


    else:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh'])))

    counter += 1


  model.add(Dense(1, activation='sigmoid'))


  LOSS_FUNCTION = "binary_crossentropy"
  OPTIMIZER = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  METRICS = ["accuracy"]

  model.compile(loss=LOSS_FUNCTION,
                optimizer=OPTIMIZER,
                metrics=METRICS)

  return model


In [33]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir2')

In [34]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 16s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 8,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 72,
 'activation1': 'relu',
 'units2': 56,
 'activation2': 'relu',
 'units3': 64,
 'activation3': 'tanh',
 'units4': 80,
 'activation4': 'relu',
 'units5': 32,
 'activation5': 'relu',
 'units6': 104,
 'activation6': 'tanh',
 'units7': 80,
 'activation7': 'tanh'}

In [36]:
model4 = tuner.get_best_models(num_models=1)[0]

In [37]:
model4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 72)                648       
                                                                 
 dense_2 (Dense)             (None, 56)                4088      
                                                                 
 dense_3 (Dense)             (None, 64)                3648      
                                                                 
 dense_4 (Dense)             (None, 80)                5200      
                                                                 
 dense_5 (Dense)             (None, 32)                2592      
                                                                 
 dense_6 (Dense)             (None, 1)                 3

In [38]:
model4.fit(X_train, y_train, epochs=10, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/10
20/20 [==============================] - 2s 14ms/step - loss: 0.4461 - accuracy: 0.7866 - val_loss: 0.5174 - val_accuracy: 0.7338
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4348 - accuracy: 0.8013 - val_loss: 0.5801 - val_accuracy: 0.7273
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4368 - accuracy: 0.7964 - val_loss: 0.5526 - val_accuracy: 0.7208
Epoch 10/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4363 - accuracy: 0.7997 - val_loss: 0.5365 - val_accuracy: 0.7468
